**[中級機械学習 ホームページ](https://www.kaggle.com/learn/intermediate-machine-learning)**

---


この演習では、**パイプライン**を使用して機械学習コードの効率を向上させます。

# セットアップ

以下の質問はあなたの作業にフィードバックを与えます。次のセルを実行してフィードバックシステムをセットアップしてください。

In [ ]:
# コードチェックのセットアップ
from learntools.core import binder
binder.bind(globals())
from learntools.ml_intermediate.ex4 import *
print("セットアップ完了")

あなたは [Kaggle Learn Users向け住宅価格競争](https://www.kaggle.com/c/home-data-for-ml-course) のデータを使って作業します。

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

次のコードセルをそのまま実行すると、`X_train`、`X_valid`、`y_train`、`y_valid` にトレーニングセットとバリデーションセットが読み込まれます。テストセットは `X_test` に読み込まれます。

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# データの読み込み
X_full = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# 目的変数が欠損している行を削除し、予測変数と目的変数を分ける
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# 訓練データからバリデーションセットを分離
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# 「カーディナリティ」とはカラム内のユニークな値の数を意味する
# 比較的カーディナリティが低いカテゴリ変数カラムを選択（便宜的・恣意的）
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# 数値カラムを選択
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# 選択したカラムのみ残す
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [ ]:
X_train.head()

次のコードセルは、チュートリアルのコードを使ってデータの前処理とモデルの学習を行っています。このコードは変更せずに実行してください。

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# 数値データの前処理
numerical_transformer = SimpleImputer(strategy='constant')

# カテゴリーデータの前処理
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 数値・カテゴリデータの前処理をまとめる
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# モデルの定義
model = RandomForestRegressor(n_estimators=100, random_state=0)

# 前処理・モデルコードをパイプラインでまとめる
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# 訓練データの前処理およびモデル学習
clf.fit(X_train, y_train)

# バリデーションデータの前処理と予測
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

このコードでは平均絶対誤差（MAE）は約17862となります。次のステップでは、より良くなるようにコードを修正します。

# ステップ1: パフォーマンスの向上

### パートA

今度はあなたの番です！下のコードセルで自分自身の前処理手順とランダムフォレストモデルを定義してください。次の変数について値を埋めてください。
- `numerical_transformer`
- `categorical_transformer`
- `model`

この演習を通過するには、有効な前処理ステップとランダムフォレストモデルを定義するだけでOKです。

In [ ]:
# 数値データの前処理
numerical_transformer = SimpleImputer(strategy='median') # ここにあなたのコード

# カテゴリデータの前処理
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
]) # ここにあなたのコード

# 数値・カテゴリデータの前処理をまとめる
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# モデルの定義
model = RandomForestRegressor(n_estimators=200, random_state=0) # ここにあなたのコード

# 解答チェック
step_1.a.check()

In [ ]:
# 下の行でヒントや解答コードを見ることができます
#step_1.a.hint()
#step_1.a.solution()

### パートB

下のコードセルはそのまま実行してください。

このステップに合格するには、**パートA** でMAEが上記のコードよりも低くなるパイプラインを構築する必要があります。ここではじっくりと、MAE をどこまで下げられるか、いろんなアプローチを試してみてください！（もし合格しなければ、パートAで前処理やモデルを変更してください。）

In [ ]:
# 前処理とモデリングのコードをパイプラインでまとめる
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# 訓練データの前処理及びモデル学習
my_pipeline.fit(X_train, y_train)

# バリデーションデータの前処理と予測
preds = my_pipeline.predict(X_valid)

# モデルを評価
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

# 解答チェック
step_1.b.check()

In [ ]:
# 下の行でヒントを見ることができます
#step_1.b.hint()

# ステップ2: テスト予測の作成

次は、学習済みモデルを使ってテストデータで予測を作成します。

In [ ]:
# テストデータの前処理および予測
preds_test = my_pipeline.predict(X_test) # ここにあなたのコード

# 解答チェック
step_2.check()

In [ ]:
# 下の行でヒントや解答コードを見ることができます
#step_2.hint()
#step_2.solution()

次のコードセルを変更せずに実行して、結果をコンペに直接提出可能なCSVファイルとして保存しましょう。

In [ ]:
# テスト予測をファイルに保存
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

# ステップ3: 結果を提出しましょう

ステップ2が正常に完了したら、リーダーボードへの提出準備が整いました！提出する場合は、必ず[このリンク](https://www.kaggle.com/c/home-data-for-ml-course)の **Join Competition** ボタンをクリックして既にコンペに参加しておいてください。
- まず右上の青い **COMMIT** ボタンをクリックしてください。ポップアップウィンドウが表示されます。
- コードの実行が完了後、ポップアップウィンドウ右上の青い **Open Version** ボタンをクリックします。これで同じページの閲覧モードになります。再びこれらの指示に戻るにはスクロールダウンしてください。
- 画面左側の **Output** タブをクリックし、**Submit to Competition** ボタンを押してリーダーボードへ結果を提出します。
- さらにスコア向上に取り組みたい場合は、画面右上の青色 **Edit** ボタンを選択してください。モデルを変更し、同じプロセスを繰り返せます。

# 続きを学びましょう

[**クロスバリデーション**](https://www.kaggle.com/alexisbcook/cross-validation) について学びましょう。これはモデルパフォーマンスをより正確に評価するための手法です！

---
**[中級機械学習 ホームページ](https://www.kaggle.com/learn/intermediate-machine-learning)**





*質問やコメントがありますか？[Learning Discussion フォーラム](https://www.kaggle.com/learn-forum) で他の学習者とチャットしましょう。*